In [6]:
import pickle
from flask import Flask, jsonify, request
import pandas as pd
import joblib

In [7]:
file_name = 'best_model.pkl'
port = 5000

In [9]:
#importing necessary libraries

app = Flask(__name__)

@app.route("/predict", methods=['POST'])
def do_prediction():
    #loading saved model here in this python file
    unpickler = pickle.Unpickler(open(file_name, 'rb'))
    model = unpickler.load()

    #creating data frame of JSON data
    json = request.get_json()
    train = pd.DataFrame(json, index=[0])
    # predict
    y_predict = model.predict(train)
    res= {"Predicted note of rmse" : y_predict[0]}
    return jsonify(res)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.137:5001
Press CTRL+C to quit


In [23]:
###arabi try
from flask import Flask, jsonify, request
import pandas as pd
import pickle

app = Flask(__name__)

# Define the custom function in the global scope
def simple_imputer(X):
    return X.fillna(X.median())
def scale(X):
    std_dev = X.std()
    if (std_dev == 0).all():
        return X - X.mean()
    return (X - X.mean()) / std_dev

# Load the model after the function definition
file_name = 'best_model.pkl'
with open(file_name, 'rb') as f:
    model = pickle.load(f)

@app.route("/predict", methods=['POST'])
def do_prediction():
    try:
        # Creating DataFrame from JSON data
        data = request.get_json()
        df = pd.DataFrame(data, index=[0])

        # Predict
        y_predict = model.predict(df)
        res = {"Predicted note of rmse": y_predict[0]}
        return jsonify(res)
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == "__main__":
    port = 5000
    app.run(host='0.0.0.0', port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.137:5000
Press CTRL+C to quit


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import pickle
import gzip
import json

books_clean = pd.read_csv('books.csv',error_bad_lines=False,warn_bad_lines=False)
books_clean['numberOfAuthors'] = books_clean['authors'].str.count('/')+1
## fixing the two dates
date_mapping = {
    '11/31/2000': '12/01/2001',
    '6/31/1982': '07/01/1982'
}
books_clean['publication_date'] = books_clean['publication_date'].replace(date_mapping)

  
books_clean['publication_date'] = pd.to_datetime(books_clean['publication_date'])

    
books_clean['day'] = books_clean['publication_date'].dt.day
books_clean['month'] = books_clean['publication_date'].dt.month
books_clean['year'] = books_clean['publication_date'].dt.year
    
def check_book_type(title):
    
    keywords = ['set', 'collection', 'boxed', 'volume', 'volumes', 'trilogy', '#1-', '1-3', '1-4', '1-5', '1-6', '1-7', '1-8', 'volume i', 'volume ii', '1-', '2-', '3-', '4-', '5-', '6-', '7-', '8-', 'series', '/']
    for keyword in keywords:
        if keyword in title.lower():
            return '1'
    return '0'
books_clean['bookType'] = books_clean['title'].apply(check_book_type)
language_counts = books_clean['language_code'].value_counts()
books_clean['lang_code_new'] = books_clean['language_code'].apply(lambda x: 'others' if language_counts[x] < 15 else x)
median_value = books_clean[books_clean['  num_pages'] != 0]['  num_pages'].median()
books_clean['  num_pages'] = books_clean['  num_pages'].replace(0, median_value)
json_data_list = []

# Open the Gzip file in binary mode for reading
with gzip.open('goodreads_book_genres_initial.json.gz', 'rb') as json_file:
    for line in json_file:
        # Decode each line and load the JSON
        json_data = json.loads(line.decode('utf-8'))
        json_data_list.append(json_data)

# Convert the list of JSON objects into a DataFrame
json_df = pd.DataFrame(json_data_list)

books_clean.rename(columns={'bookID': 'book_id'}, inplace=True)
json_df['book_id'] = json_df['book_id'].astype(int)
books_clean_merged = pd.merge(books_clean, json_df, on="book_id", how="left")
def get_max_key(d):
    if isinstance(d, dict): 
        if d:  
            return max(d, key=d.get)
    return None  

books_clean_merged['type'] = books_clean_merged['genres'].apply(get_max_key)
books_clean_merged['type'].fillna('others', inplace=True)
books_clean_merged['collaboration'] = books_clean_merged['numberOfAuthors'].apply(lambda x: 'Yes' if x > 1 else 'No')
publisher_counts = books_clean_merged['publisher'].value_counts()


def classify_publisher(count):
    if count < 15:
        return 'Rare'
    elif count <= 100:
        return 'Medium'
    else:
        return 'Famous'
    
books_clean_merged['publisher_class'] = books_clean_merged['publisher'].map(publisher_counts).apply(classify_publisher)
columns_to_remove = ['average_rating','publisher','book_id', 'title', 'authors','isbn', 'isbn13','language_code','publication_date','numberOfAuthors','genres']

train= books_clean_merged.drop(columns=columns_to_remove)
train = train.rename(columns={'  num_pages': 'num_pages'})

In [ ]:
import requests

url = 'http://localhost:5000/predict'
dict_x = dict_x = train.iloc[0].to_dict()
r = requests.post(url, json=dict_x)
print(r.json())
